In [1]:
import numpy as np
from collections import deque
import heapq

In [2]:
class State:
    def __init__(self, position, speed, direction, moves=0, parent=None, visited_directions=None):
      self.position = position  # (row, col)
      self.speed = speed        # Current speed
      self.direction = direction  # 0: right, 1: down, 2: left, 3: up
      self.moves = moves        # Number of moves taken
      self.parent = parent      # Parent state for path reconstruction
      self.visited_directions = visited_directions or set()  # Directions visited so far
    def __lt__(self, other):
      return self.moves < other.moves

    def __eq__(self, other):
      return (self.position == other.position and
              self.speed == other.speed and
              self.direction == other.direction and
              self.visited_directions == other.visited_directions)
    def __hash__(self):
      return hash((self.position, self.speed, self.direction, frozenset(self.visited_directions)))


In [3]:
def parse_matrix():
  matrix = [
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 2, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 2, 1],
       [1, 2, 0, 0, 1, 1, 1, 0, 0, 1],
       [1, 0, 0, 0, 1, 1, 1, 0, 0, 1],
       [1, 3, 0, 0, 1, 1, 1, 0, 0, 1],
       [1, 0, 0, 0, 1, 1, 2, 2, 0, 1],
       [1, 0, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 0, 2, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 2, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  ]
  return np.array(matrix)

In [4]:
def find_start_position(matrix):
  """Find and return the starting position marked by 3 in the matrix"""
  start_pos = None
  for i in range(len(matrix)):
      for j in range(len(matrix[0])):
          if matrix[i][j] == 3:
              start_pos = (i, j)
              # Reset the start position to 0 for path finding
              matrix[i][j] = 0
              return start_pos
  return None


In [5]:
def is_valid_position(matrix, position):
    """Check if a position is valid (within bounds and not an obstacle or lane)"""
    row, col = position
    if (0 <= row < len(matrix) and
        0 <= col < len(matrix[0]) and
        matrix[row][col] != 1 and
        matrix[row][col] != 2):
        return True
    return False

In [6]:
def is_path_clear(matrix, start_pos, end_pos):
    """Check if the path from start_pos to end_pos is clear (no obstacles or lane crossings)"""
    start_row, start_col = start_pos
    end_row, end_col = end_pos

    # Get range of positions to check
    min_row, max_row = min(start_row, end_row), max(start_row, end_row)
    min_col, max_col = min(start_col, end_col), max(start_col, end_col)

    for r in range(min_row, max_row + 1):
        for c in range(min_col, max_col + 1):
            if matrix[r][c] == 1 or matrix[r][c] == 2:
                return False
    return True

In [8]:
def get_next_states(current_state, matrix):
    """Generate all valid next states from the current state"""
    next_states = []
    row, col = current_state.position
    speed = current_state.speed
    direction = current_state.direction
    visited_dirs = current_state.visited_directions.copy()
    visited_dirs.add(direction)
    # Defining direction vectors (right, down, left, up)
    dir_vectors = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    current_dir_vector = dir_vectors[direction]
    # Case 1: Continue with same direction, adjust speed
    for delta_speed in [-1, 0, 1]:
        new_speed = speed + delta_speed
        if new_speed < 0:
            continue
        # Applying movement based on direction and speed
        new_row = row + current_dir_vector[0] * new_speed
        new_col = col + current_dir_vector[1] * new_speed
        new_pos = (new_row, new_col)
        if is_valid_position(matrix, new_pos) and is_path_clear(matrix, (row, col), new_pos):
            next_state = State(new_pos, new_speed, direction,
                              current_state.moves + 1, current_state, visited_dirs)
            next_states.append(next_state)
    # Case 2: If speed is 0 or 1, we can change direction
    if speed <= 1:
        #  changing direction to the next one clockwise (turn right)
        new_direction = (direction + 1) % 4
        new_dir_vector = dir_vectors[new_direction]

        for new_speed in [0, 1]:
            new_row = row + new_dir_vector[0] * new_speed
            new_col = col + new_dir_vector[1] * new_speed
            new_pos = (new_row, new_col)
            if is_valid_position(matrix, new_pos) and is_path_clear(matrix, (row, col), new_pos):
                next_state = State(new_pos, new_speed, new_direction,
                                  current_state.moves + 1, current_state, visited_dirs)
                next_states.append(next_state)

    return next_states


In [9]:
def has_completed_clockwise_round(start_pos, current_pos, visited_directions):
    """Check if we've completed a clockwise round"""
    if start_pos == current_pos and len(visited_directions) >= 4:
        # We've visited all 4 directions and returned to start
        return True
    return False

In [10]:
def reconstruct_path(final_state, matrix, start_pos):
    """Reconstruct the path from the final state and mark it on the matrix"""
    path = []
    current = final_state

    while current is not None:
        path.append(current.position)
        current = current.parent
    path.reverse()
    # Creating a result matrix marking the path
    result_matrix = matrix.copy()
    for pos in path:
        result_matrix[pos] = 3

    # Ensuring the start position is marked as 3
    result_matrix[start_pos] = 3

    return result_matrix, len(path) - 1  # -1 because initial position isn't a move


In [11]:
def find_minimum_moves(matrix):
    """Find the minimum number of moves to complete a clockwise round"""
    # Find start position
    start_pos = find_start_position(matrix)
    if not start_pos:
        print("No starting position found!")
        return None, 0
    priority_queue = []

    # Creating initial state (position, speed=0, direction=0, moves=0)
    initial_state = State(start_pos, 0, 0, 0)
    heapq.heappush(priority_queue, (0, initial_state))

    visited = set()
    while priority_queue:
        _, current_state = heapq.heappop(priority_queue)

        # Skip if we've already visited this state
        state_key = (current_state.position, current_state.speed, current_state.direction,
                    frozenset(current_state.visited_directions))
        if state_key in visited:
            continue

        visited.add(state_key)

        # Checking if we've completed a round and returned to start with speed 0
        if (current_state.position == start_pos and
            current_state.speed == 0 and
            has_completed_clockwise_round(start_pos, current_state.position, current_state.visited_directions)):
            return reconstruct_path(current_state, matrix, start_pos)

        # to generate next possible states
        next_states = get_next_states(current_state, matrix)

        for next_state in next_states:
            next_key = (next_state.position, next_state.speed, next_state.direction,
                       frozenset(next_state.visited_directions))

            if next_key not in visited:
                # Calculating heuristic: Manhattan distance to start + penalty for high speed
                # + penalty for not having visited all directions
                dr, dc = abs(next_state.position[0] - start_pos[0]), abs(next_state.position[1] - start_pos[1])
                direction_penalty = 4 - len(next_state.visited_directions)
                heuristic = dr + dc + next_state.speed + direction_penalty * 2

                priority = next_state.moves + heuristic
                heapq.heappush(priority_queue, (priority, next_state))

    print("No solution found!")
    return matrix, 0

In [12]:
def print_matrix(matrix):
    """Print the matrix in a readable format"""
    for row in matrix:
        print(' '.join(map(str, row)))

def solve_race_track():
    """Main function to solve the race track problem"""
    # Load the matrix
    matrix = parse_matrix()
    print("Original Matrix:")
    print_matrix(matrix)
    result_matrix, moves = find_minimum_moves(matrix)

    print("\nPath Matrix:")
    print_matrix(result_matrix)
    print(f"\nMinimum number of moves: {moves}")
    return result_matrix

In [13]:
if __name__ == "__main__":
    solve_race_track()

Original Matrix:
1 1 1 1 1 1 1 1 1 1
1 0 0 0 0 0 0 0 0 1
1 0 0 2 0 0 0 0 0 1
1 0 0 0 0 0 0 0 2 1
1 2 0 0 1 1 1 0 0 1
1 0 0 0 1 1 1 0 0 1
1 3 0 0 1 1 1 0 0 1
1 0 0 0 1 1 2 2 0 1
1 0 0 0 1 1 0 0 0 1
1 0 0 0 1 1 0 0 0 1
1 0 2 0 0 0 0 0 0 1
1 0 0 0 0 0 2 0 0 1
1 0 0 0 0 0 0 0 0 1
1 1 1 1 1 1 1 1 1 1

Path Matrix:
1 1 1 1 1 1 1 1 1 1
1 0 0 0 0 0 0 0 0 1
1 0 0 2 0 0 0 0 0 1
1 0 0 0 0 0 0 0 2 1
1 2 0 0 1 1 1 0 0 1
1 0 0 0 1 1 1 0 0 1
1 3 0 0 1 1 1 0 0 1
1 0 0 0 1 1 2 2 0 1
1 0 0 0 1 1 0 0 0 1
1 0 0 0 1 1 0 0 0 1
1 0 2 0 0 0 0 0 0 1
1 0 0 0 0 0 2 0 0 1
1 0 0 0 0 0 0 0 0 1
1 1 1 1 1 1 1 1 1 1

Minimum number of moves: 4
